In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# --------------------------------------------------
# PASO 1: Cargar el DataFrame ya procesado
# --------------------------------------------------
df = pd.read_csv("../../data/processed/navegacion_clientes_experimento_limpio.csv")

# --------------------------------------------------
# PASO 2: Definir si el cliente completó el proceso (llegó a 'confirm')
# --------------------------------------------------
# Para cada registro, asignamos 'completed' = True si process_step == "confirm", False en caso contrario.
df["completed"] = df["process_step"] == "confirm"

# --------------------------------------------------
# PASO 3: Agrupar a nivel de cliente (client_id) y variation
# --------------------------------------------------
# Cada cliente se agrupa para que, si completó al menos una sesión, se marque como True.
# Además, se conserva la variable 'calls_6_mnth' (número de llamadas en los últimos 6 meses) para evaluar el coste.
df_clients = (
    df.groupby(["variation", "client_id"], as_index=False)
      .agg({
          "completed": "max",     # 1 si el cliente completó al menos una vez
          "calls_6_mnth": "first"   # Se asume que este valor es constante por cliente
      })
)

# (Opcional) Filtrar para excluir registros con Variation desconocido
df_clients = df_clients[df_clients["variation"].isin(["Control", "Test"])].copy()

# --------------------------------------------------
# PASO 4: Definir el umbral de coste y crear la variable "cost_effective"
# --------------------------------------------------
# Definimos un umbral para el número de llamadas; por ejemplo, se considera rentable si el cliente
# completó el proceso y realizó <= 3 llamadas en los últimos 6 meses.
threshold_calls = 3

df_clients["cost_effective"] = (df_clients["completed"]) & (df_clients["calls_6_mnth"] <= threshold_calls)

# --------------------------------------------------
# PASO 5: Calcular el Cost-Effectiveness Rate (CE Rate) por grupo (a nivel de cliente)
# --------------------------------------------------
# Agrupar por 'variation' para obtener la tasa de éxito rentable: la media de 'cost_effective'
ce_data = df_clients.groupby("variation")["cost_effective"].agg(["mean", "sum", "count"])

print("=== COST-EFFECTIVENESS RATE (CE) POR GRUPO (a nivel de cliente) ===")
print(ce_data)
print("\n- mean  = Tasa de éxito rentable (CE Rate)")
print("- sum   = Nº de clientes que completaron de forma rentable")
print("- count = Nº total de clientes en cada grupo\n")

# --------------------------------------------------
# PASO 6: Test de hipótesis (Z-test de proporciones) para comparar CE Rate entre Test y Control
# --------------------------------------------------
test_sum   = ce_data.loc["Test", "sum"]
test_count = ce_data.loc["Test", "count"]
ctrl_sum   = ce_data.loc["Control", "sum"]
ctrl_count = ce_data.loc["Control", "count"]

count = np.array([test_sum, ctrl_sum])
nobs  = np.array([test_count, ctrl_count])

stat, p_value = sm.stats.proportions_ztest(count, nobs, alternative='two-sided')
alpha = 0.05

print("=== PRUEBA DE HIPÓTESIS PARA COST-EFFECTIVENESS (a nivel de cliente) ===")
print(f"Estadístico z: {stat:.4f}")
print(f"p-value      : {p_value:.6f}")

if p_value < alpha:
    print(f"\nConclusión: Rechazamos H0 (p < {alpha}).")
    print("Existe una diferencia estadísticamente significativa en la Cost-Effectiveness Rate entre Test y Control.")
else:
    print(f"\nConclusión: No rechazamos H0 (p >= {alpha}).")
    print("No se detecta diferencia estadísticamente significativa en la Cost-Effectiveness Rate entre Test y Control.")

# --------------------------------------------------
# PASO 7: Resumen final
# --------------------------------------------------
ce_test    = ce_data.loc["Test", "mean"]
ce_control = ce_data.loc["Control", "mean"]

print(f"\n=== Resumen Final ===")
print(f"Cost-Effective Rate (Test)    : {ce_test:.3%}")
print(f"Cost-Effective Rate (Control) : {ce_control:.3%}")
print(f"p-value                       : {p_value:.6f}")


=== COST-EFFECTIVENESS RATE (CE) POR GRUPO (a nivel de cliente) ===
               mean   sum  count
variation                       
Control    0.412244  6734  16335
Test       0.443184  8362  18868

- mean  = Tasa de éxito rentable (CE Rate)
- sum   = Nº de clientes que completaron de forma rentable
- count = Nº total de clientes en cada grupo

=== PRUEBA DE HIPÓTESIS PARA COST-EFFECTIVENESS (a nivel de cliente) ===
Estadístico z: 5.8497
p-value      : 0.000000

Conclusión: Rechazamos H0 (p < 0.05).
Existe una diferencia estadísticamente significativa en la Cost-Effectiveness Rate entre Test y Control.

=== Resumen Final ===
Cost-Effective Rate (Test)    : 44.318%
Cost-Effective Rate (Control) : 41.224%
p-value                       : 0.000000


## Explicación del Código

### Carga y definición de completed

Se carga el DataFrame limpio y se crea una columna booleana completed que es True si el process_step es "confirm".

### Agrupación a nivel de cliente

Se agrupa el DataFrame por variation y client_id, usando max() en completed (así, si el cliente completó al menos una sesión, se considera exitoso) y se conserva calls_6_mnth.

### Definición del umbral de coste

Se establece un umbral (en este caso, 3 llamadas). Un cliente se considera "cost-effective" si completó el proceso y sus llamadas no superan este umbral.

### Cálculo de la tasa de éxito rentable (CE Rate)

Se agrupa por variation y se calcula la media de la variable cost_effective.